In [ ]:
%load_ext autoreload
%autoreload 5

In [ ]:
import json
import numpy as np
import pandas as pd
import random

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    plot_confusion_matrix,
)

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from weight_lifting import WeightLifting
from sfs_features import (
    get_lr_features,
    get_svm_features,
    get_mpl_features
)

In [ ]:
np.random.seed(42)
random.seed(42)

### BASE PADRAO

In [ ]:
engine = WeightLifting()
df = engine.load_df()
df = engine.transform(df)
X_train, X_test, y_train, y_test = engine.create_train_test(df)
base_padrao_predict = engine.fit_and_predict(X_train, X_test, y_train, y_test)
#engine.plot_results(base_padrao_predict, X_test, y_test)

sfs_predict = []

### FEATURE SELECTION WITH SequentialFeatureSelector 

In [ ]:
# LOGISTIC REGRESSION
lr_X_train, lr_X_test, lr_y_train, lr_y_test = engine.create_train_test(df, features=get_mpl_features())
lr_predict = engine.fit_and_predict(lr_X_train, lr_X_test, lr_y_train, lr_y_test)
lr_predict = lr_predict[0:1][0] # Somente o LR interessa
sfs_predict.append(lr_predict)

# SUPORT VECTOR MACHINES
svm_X_train, svm_X_test, svm_y_train, svm_y_test = engine.create_train_test(df, features=get_svm_features())
svm_predict = engine.fit_and_predict(svm_X_train, svm_X_test, svm_y_train, svm_y_test)
svm_predict = svm_predict[1:2][0] # Somente o SVM interessa
sfs_predict.append(svm_predict)

# MULTILAYER PERCEPTRON
svm_X_train, svm_X_test, svm_y_train, svm_y_test = engine.create_train_test(df, features=get_mpl_features())
mpl_predict = engine.fit_and_predict(svm_X_train, svm_X_test, svm_y_train, svm_y_test)
mpl_predict = mpl_predict[2:3][0] # Somente o mpl interessa
sfs_predict.append(mpl_predict)

### COMPARACAO DE RESULTADOS

In [ ]:
# Unificando os resultados em um unico DF
resultados_inicial = pd.DataFrame(base_padrao_predict)
resultados_inicial['ESTADO'] =  'INICIAL'

resultados_sfs = pd.DataFrame(sfs_predict)
resultados_sfs['ESTADO'] =  'SFS'

resultados = pd.concat([resultados_inicial,resultados_sfs])
resultados['NAME_ESTADO'] = resultados['name'] + '_' + resultados['ESTADO']

In [ ]:
#round(accuracy_score(y_test, predict), 4)
resultados['accuracy_score'] = resultados.apply(
    lambda x: 
        round(accuracy_score(y_test, x['predict']), 4), axis=1)

resultados['f1_score'] = resultados.apply(
    lambda x: 
        round(f1_score(y_test, x['predict'], average='macro'), 4), axis=1)

resultados['precision_score'] = resultados.apply(
    lambda x: 
        round(precision_score(y_test, x['predict'], average='macro'), 4), axis=1)

resultados['recall_score'] = resultados.apply(
    lambda x: 
        round(recall_score(y_test, x['predict'], average='macro'), 4), axis=1)     
        
resultados.sort_values(by='name')

In [ ]:
resultados

In [ ]:
data = [[30, 25, 50, 20],
[40, 23, 51, 17],
[35, 22, 45, 19]]
X = np.arange(4)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, data[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, data[1], color = 'g', width = 0.25)
ax.bar(X + 0.50, data[2], color = 'r', width = 0.25)